In [5]:
from __future__ import division
import pandas as pd
import numpy as np
import operator
import pickle

In [6]:
df = pd.read_csv("data/nodes.tsv", sep='\t')

In [7]:
df

,ID,Name,Depth
0,HP:0000001,All,0
1,HP:0012823,Clinical modifier,1
2,HP:0000118,Phenotypic abnormality,1
3,HP:0032223,Blood group,1
4,HP:0031797,Clinical course,1
...,...,...,...
15242,HP:0009952,Complete duplication of the middle phalanx of ...,15
15243,HP:0009951,Partial duplication of the distal phalanx of t...,15
15244,HP:0009979,Complete duplication of the middle phalanx of ...,15
15245,HP:0009978,Complete duplication of the distal phalanx of ...,15


In [8]:
with open('graph_ontology.pkl', 'rb') as input:
    graph_load = pickle.load(input)

In [9]:
df.loc[df['ID'] == 'HP:0012824']

,ID,Name,Depth
76,HP:0012824,Severity,2


In [10]:
Phenotypic_abnor_descendants = graph_load.get_descendants('HP:0000118')
df_new = df[df.ID.isin(Phenotypic_abnor_descendants)]
entry = df.loc[df['ID'] == 'HP:0000118']
df_new = pd.concat([entry,df_new])
df_new = df_new.rename(columns = {'ID': 'HPO_ID', 'Name': 'Description'}, inplace = False)

In [11]:
df_new

,HPO_ID,Description,Depth
2,HP:0000118,Phenotypic abnormality,1
9,HP:0000478,Abnormality of the eye,2
11,HP:0000769,Abnormality of the breast,2
12,HP:0001574,Abnormality of the integument,2
13,HP:0003011,Abnormality of the musculature,2
...,...,...,...
15242,HP:0009952,Complete duplication of the middle phalanx of ...,15
15243,HP:0009951,Partial duplication of the distal phalanx of t...,15
15244,HP:0009979,Complete duplication of the middle phalanx of ...,15
15245,HP:0009978,Complete duplication of the distal phalanx of ...,15


In [12]:
df_new.to_csv("Phenotypic_descendants_depth.csv", index= False)

In [13]:
phenotype_data = pd.read_csv("Phenotypic_descendants_depth.csv")
phenotype_data

,HPO_ID,Description,Depth
0,HP:0000118,Phenotypic abnormality,1
1,HP:0000478,Abnormality of the eye,2
2,HP:0000769,Abnormality of the breast,2
3,HP:0001574,Abnormality of the integument,2
4,HP:0003011,Abnormality of the musculature,2
...,...,...,...
15024,HP:0009952,Complete duplication of the middle phalanx of ...,15
15025,HP:0009951,Partial duplication of the distal phalanx of t...,15
15026,HP:0009979,Complete duplication of the middle phalanx of ...,15
15027,HP:0009978,Complete duplication of the distal phalanx of ...,15


In [14]:
df_new1 = phenotype_data.loc[phenotype_data.Depth<3][['HPO_ID', 'Description']]

In [15]:
df_new1

,HPO_ID,Description
0,HP:0000118,Phenotypic abnormality
1,HP:0000478,Abnormality of the eye
2,HP:0000769,Abnormality of the breast
3,HP:0001574,Abnormality of the integument
4,HP:0003011,Abnormality of the musculature
5,HP:0025354,Abnormal cellular phenotype
6,HP:0002715,Abnormality of the immune system
7,HP:0002664,Neoplasm
8,HP:0040064,Abnormality of limbs
9,HP:0000152,Abnormality of head or neck


In [16]:
Immediate_descendants_list = graph_load.get_immediate_descendants('HP:0000118')
Immediate_descendants_list

{'HP:0000119',
 'HP:0000152',
 'HP:0000478',
 'HP:0000598',
 'HP:0000707',
 'HP:0000769',
 'HP:0000818',
 'HP:0000924',
 'HP:0001197',
 'HP:0001507',
 'HP:0001574',
 'HP:0001608',
 'HP:0001626',
 'HP:0001871',
 'HP:0001939',
 'HP:0002086',
 'HP:0002664',
 'HP:0002715',
 'HP:0003011',
 'HP:0003549',
 'HP:0025031',
 'HP:0025142',
 'HP:0025354',
 'HP:0040064',
 'HP:0041054',
 'HP:0041158',
 'HP:0041161',
 'HP:0045027'}

In [17]:
result = pd.DataFrame(columns=['HPO_ID','Definition','Ancestor'])
i=0
for index, row in phenotype_data.iloc[29:].iterrows():
    ancestors_list = graph_load.get_ancestors(row['HPO_ID'])
    common_ancestors = ancestors_list.intersection(Immediate_descendants_list)
    if len(common_ancestors) ==1:
        print(row['HPO_ID'], ancestors_list, common_ancestors)
        try:
            result.loc[i, ['Definition']] = graph_load.nodes[row['HPO_ID']]['def']
            str_ancestors = " ".join(common_ancestors)
            result.loc[i, ['Ancestor']] = str_ancestors
            result.loc[i, ['HPO_ID']] = row['HPO_ID']
            i = i+1
        except KeyError:
            pass

HP:0025461 {'HP:0000118', 'HP:0025354', 'HP:0000001', 'HP:0025461'} {'HP:0025354'}
HP:0031352 {'HP:0000118', 'HP:0025142', 'HP:0000001', 'HP:0031352'} {'HP:0025142'}
HP:0000464 {'HP:0000464', 'HP:0000118', 'HP:0000001', 'HP:0000152'} {'HP:0000152'}
HP:0001686 {'HP:0000118', 'HP:0001686', 'HP:0000001', 'HP:0001608'} {'HP:0001608'}
HP:0031863 {'HP:0000118', 'HP:0031863', 'HP:0000001', 'HP:0001939'} {'HP:0001939'}
HP:0001928 {'HP:0000118', 'HP:0000001', 'HP:0001871', 'HP:0001928'} {'HP:0001871'}
HP:0009088 {'HP:0000118', 'HP:0000001', 'HP:0009088', 'HP:0001608'} {'HP:0001608'}
HP:0041079 {'HP:0041079', 'HP:0000118', 'HP:0000001', 'HP:0001507'} {'HP:0001507'}
HP:0012379 {'HP:0000118', 'HP:0000001', 'HP:0012379', 'HP:0001939'} {'HP:0001939'}
HP:0011024 {'HP:0025031', 'HP:0000118', 'HP:0000001', 'HP:0011024'} {'HP:0025031'}
HP:0100685 {'HP:0000118', 'HP:0003549', 'HP:0000001', 'HP:0100685'} {'HP:0003549'}
HP:0031838 {'HP:0000118', 'HP:0000001', 'HP:0031838', 'HP:0001939'} {'HP:0001939'}
HP:0

HP:0012420 {'HP:0000001', 'HP:0000118', 'HP:0012420', 'HP:0001197', 'HP:0001560'} {'HP:0001197'}
HP:0046506 {'HP:0012531', 'HP:0000001', 'HP:0000118', 'HP:0025142', 'HP:0046506'} {'HP:0025142'}
HP:0030157 {'HP:0012531', 'HP:0000001', 'HP:0000118', 'HP:0030157', 'HP:0025142'} {'HP:0025142'}
HP:0011875 {'HP:0000001', 'HP:0001871', 'HP:0011875', 'HP:0000118', 'HP:0001872'} {'HP:0001871'}
HP:0041053 {'HP:0000001', 'HP:0000152', 'HP:0000118', 'HP:0041053', 'HP:0000234'} {'HP:0000152'}
HP:0040127 {'HP:0000001', 'HP:0001939', 'HP:0040127', 'HP:0000118', 'HP:0012337'} {'HP:0001939'}
HP:0031062 {'HP:0000001', 'HP:0031062', 'HP:0000118', 'HP:0025142', 'HP:0031058'} {'HP:0025142'}
HP:0001311 {'HP:0000001', 'HP:0000707', 'HP:0000118', 'HP:0001311', 'HP:0012638'} {'HP:0000707'}
HP:0100886 {'HP:0000001', 'HP:0012372', 'HP:0000118', 'HP:0100886', 'HP:0000478'} {'HP:0000478'}
HP:0012021 {'HP:0001626', 'HP:0000001', 'HP:0012021', 'HP:0010948', 'HP:0000118'} {'HP:0001626'}
HP:0020071 {'HP:0020071', 'HP:

HP:0002641 {'HP:0000001', 'HP:0001871', 'HP:0002641', 'HP:0001977', 'HP:0000118'} {'HP:0001871'}
HP:0010947 {'HP:0001626', 'HP:0000001', 'HP:0010947', 'HP:0010948', 'HP:0000118'} {'HP:0001626'}
HP:0031087 {'HP:0000001', 'HP:0031087', 'HP:0000118', 'HP:0001507', 'HP:0001510'} {'HP:0001507'}
HP:0000970 {'HP:0000001', 'HP:0025276', 'HP:0001574', 'HP:0000970', 'HP:0000118'} {'HP:0001574'}
HP:0012533 {'HP:0012531', 'HP:0000001', 'HP:0000118', 'HP:0025142', 'HP:0012533'} {'HP:0025142'}
HP:0011902 {'HP:0000001', 'HP:0001871', 'HP:0011902', 'HP:0001877', 'HP:0000118'} {'HP:0001871'}
HP:0012700 {'HP:0025032', 'HP:0000001', 'HP:0025031', 'HP:0000118', 'HP:0012700'} {'HP:0025031'}
HP:0005521 {'HP:0000001', 'HP:0001871', 'HP:0001977', 'HP:0000118', 'HP:0005521'} {'HP:0001871'}
HP:0031965 {'HP:0000001', 'HP:0001871', 'HP:0001877', 'HP:0000118', 'HP:0031965'} {'HP:0001871'}
HP:0030687 {'HP:0000001', 'HP:0003117', 'HP:0000818', 'HP:0000118', 'HP:0030687'} {'HP:0000818'}
HP:0100242 {'HP:0011792', 'HP:

HP:0002093 {'HP:0002795', 'HP:0000001', 'HP:0000118', 'HP:0002086', 'HP:0002093'} {'HP:0002086'}
HP:0002832 {'HP:0002832', 'HP:0000001', 'HP:0000118', 'HP:0000924', 'HP:0010766'} {'HP:0000924'}
HP:0005502 {'HP:0000001', 'HP:0001871', 'HP:0005502', 'HP:0020054', 'HP:0000118'} {'HP:0001871'}
HP:0002585 {'HP:0000001', 'HP:0002585', 'HP:0025031', 'HP:0000118', 'HP:0002012'} {'HP:0025031'}
HP:0008226 {'HP:0000001', 'HP:0008226', 'HP:0000818', 'HP:0000118', 'HP:0008373'} {'HP:0000818'}
HP:0032510 {'HP:0012531', 'HP:0000001', 'HP:0032510', 'HP:0000118', 'HP:0025142'} {'HP:0025142'}
HP:0003258 {'HP:0000001', 'HP:0003258', 'HP:0001939', 'HP:0000118', 'HP:0012379'} {'HP:0001939'}
HP:0100016 {'HP:0000001', 'HP:0100016', 'HP:0025031', 'HP:0000118', 'HP:0002012'} {'HP:0025031'}
HP:0031187 {'HP:0000001', 'HP:0003117', 'HP:0000818', 'HP:0000118', 'HP:0031187'} {'HP:0000818'}
HP:0030684 {'HP:0000001', 'HP:0003117', 'HP:0000818', 'HP:0000118', 'HP:0030684'} {'HP:0000818'}
HP:0045040 {'HP:0000001', 'HP:

HP:0001901 {'HP:0001901', 'HP:0000001', 'HP:0001871', 'HP:0001877', 'HP:0000118'} {'HP:0001871'}
HP:0010313 {'HP:0031093', 'HP:0000001', 'HP:0010313', 'HP:0000118', 'HP:0000769'} {'HP:0000769'}
HP:0025278 {'HP:0000001', 'HP:0025276', 'HP:0025278', 'HP:0001574', 'HP:0000118'} {'HP:0001574'}
HP:0031340 {'HP:0025354', 'HP:0000001', 'HP:0000118', 'HP:0025461', 'HP:0031340'} {'HP:0025354'}
HP:0007550 {'HP:0000001', 'HP:0025276', 'HP:0007550', 'HP:0001574', 'HP:0000118'} {'HP:0001574'}
HP:0011726 {'HP:0001626', 'HP:0000001', 'HP:0011726', 'HP:0010948', 'HP:0000118'} {'HP:0001626'}
HP:0033050 {'HP:0012531', 'HP:0000001', 'HP:0000118', 'HP:0025142', 'HP:0033050'} {'HP:0025142'}
HP:0010727 {'HP:0000001', 'HP:0012372', 'HP:0010727', 'HP:0000118', 'HP:0000478'} {'HP:0000478'}
HP:0011412 {'HP:0001787', 'HP:0000001', 'HP:0000118', 'HP:0001197', 'HP:0011412'} {'HP:0001197'}
HP:0010970 {'HP:0000001', 'HP:0001871', 'HP:0001877', 'HP:0000118', 'HP:0010970'} {'HP:0001871'}
HP:0010968 {'HP:0010968', 'HP:

...

HP:0005625 {'HP:0000001', 'HP:0009121', 'HP:0011849', 'HP:0040161', 'HP:0000118', 'HP:0000925', 'HP:0005625', 'HP:0000924', 'HP:0004348', 'HP:0000939', 'HP:0003330', 'HP:0004349', 'HP:0011842', 'HP:0003468'} {'HP:0000924'}
HP:0011594 {'HP:0030962', 'HP:0000001', 'HP:0025015', 'HP:0011587', 'HP:0000118', 'HP:0012303', 'HP:0001626', 'HP:0011594', 'HP:0001679', 'HP:0011004', 'HP:0012020', 'HP:0002597', 'HP:0030680'} {'HP:0001626'}
HP:0030822 {'HP:0000001', 'HP:0000152', 'HP:0000118', 'HP:0032039', 'HP:0000492', 'HP:0000234', 'HP:0000271', 'HP:0030822', 'HP:0030820', 'HP:0000315', 'HP:0030669'} {'HP:0000152'}
HP:0031119 {'HP:0000001', 'HP:0001627', 'HP:0001647', 'HP:0001654', 'HP:0000118', 'HP:0030680', 'HP:0031118', 'HP:0031567', 'HP:0001626', 'HP:0001646', 'HP:0031119'} {'HP:0001626'}
HP:0007333 {'HP:0100547', 'HP:0000001', 'HP:0012443', 'HP:0000707', 'HP:0000118', 'HP:0012639', 'HP:0002011', 'HP:0006872', 'HP:0002060', 'HP:0007364', 'HP:0002538', 'HP:0007333', 'HP:0002977'} {'HP:0000707

HP:0031646 {'HP:0030962', 'HP:0031646', 'HP:0000001', 'HP:0025015', 'HP:0000118', 'HP:0002617', 'HP:0001626', 'HP:0001679', 'HP:0005113', 'HP:0012727', 'HP:0004942', 'HP:0002597', 'HP:0011004', 'HP:0030680'} {'HP:0001626'}
HP:0030619 {'HP:0000001', 'HP:0030603', 'HP:0000118', 'HP:0030617', 'HP:0030619', 'HP:0004329', 'HP:0030613', 'HP:0030601', 'HP:0012372', 'HP:0030612', 'HP:0000478'} {'HP:0000478'}
HP:0500069 {'HP:0000001', 'HP:0000152', 'HP:0500069', 'HP:0000118', 'HP:0032039', 'HP:0000492', 'HP:0000234', 'HP:0000271', 'HP:0000656', 'HP:0000315', 'HP:0030669'} {'HP:0000152'}
HP:0030002 {'HP:0000001', 'HP:0000152', 'HP:0000118', 'HP:0030002', 'HP:0032039', 'HP:0000492', 'HP:0000234', 'HP:0000271', 'HP:0030001', 'HP:0000315', 'HP:0030669'} {'HP:0000152'}
HP:0005653 {'HP:0000001', 'HP:0011849', 'HP:0040160', 'HP:0000118', 'HP:0005653', 'HP:0000924', 'HP:0004348', 'HP:0000939', 'HP:0003330', 'HP:0004349', 'HP:0011842'} {'HP:0000924'}
HP:0032032 {'HP:0000001', 'HP:0032032', 'HP:0000152',

HP:0025005 {'HP:0000077', 'HP:0012575', 'HP:0012210', 'HP:0000001', 'HP:0000095', 'HP:0025006', 'HP:0000079', 'HP:0000119', 'HP:0000118', 'HP:0010935', 'HP:0025005', 'HP:0031263', 'HP:0011035'} {'HP:0000119'}
HP:0001592 {'HP:0000164', 'HP:0000001', 'HP:0006483', 'HP:0000152', 'HP:0000118', 'HP:0031816', 'HP:0000163', 'HP:0009804', 'HP:0000271', 'HP:0000153', 'HP:0001592', 'HP:0000234'} {'HP:0000152'}
HP:0031982 {'HP:0002134', 'HP:0012443', 'HP:0000001', 'HP:0000707', 'HP:0010994', 'HP:0000118', 'HP:0012639', 'HP:0002011', 'HP:0010993', 'HP:0002060', 'HP:0031982', 'HP:0100547'} {'HP:0000707'}
HP:0007001 {'HP:0001317', 'HP:0000001', 'HP:0012443', 'HP:0000707', 'HP:0007001', 'HP:0000118', 'HP:0002438', 'HP:0012639', 'HP:0002011', 'HP:0002334', 'HP:0011283', 'HP:0011282'} {'HP:0000707'}
HP:0000343 {'HP:0000159', 'HP:0000001', 'HP:0000288', 'HP:0000152', 'HP:0000118', 'HP:0000177', 'HP:0031816', 'HP:0000163', 'HP:0000271', 'HP:0000153', 'HP:0000343', 'HP:0000234'} {'HP:0000152'}
HP:0000696 

HP:0410031 {'HP:0000001', 'HP:0000152', 'HP:0000118', 'HP:0410031', 'HP:0000153', 'HP:0031816', 'HP:0000174', 'HP:0000163', 'HP:0000175', 'HP:0000202', 'HP:0000271', 'HP:0100737', 'HP:0000234'} {'HP:0000152'}
HP:0000677 {'HP:0000164', 'HP:0000001', 'HP:0006483', 'HP:0000152', 'HP:0000118', 'HP:0031816', 'HP:0000163', 'HP:0009804', 'HP:0000271', 'HP:0000153', 'HP:0000677', 'HP:0000234'} {'HP:0000152'}
HP:0006332 {'HP:0000164', 'HP:0000001', 'HP:0006483', 'HP:0000152', 'HP:0000118', 'HP:0000676', 'HP:0031816', 'HP:0006332', 'HP:0000163', 'HP:0000271', 'HP:0000153', 'HP:0011064', 'HP:0011069', 'HP:0000234'} {'HP:0000152'}
HP:0012678 {'HP:0012678', 'HP:0002134', 'HP:0012443', 'HP:0000001', 'HP:0000707', 'HP:0000118', 'HP:0045007', 'HP:0012639', 'HP:0002011', 'HP:0010993', 'HP:0002060', 'HP:0002418', 'HP:0012675', 'HP:0100547'} {'HP:0000707'}
HP:0000322 {'HP:0000159', 'HP:0000001', 'HP:0000288', 'HP:0000152', 'HP:0000118', 'HP:0000177', 'HP:0031816', 'HP:0000163', 'HP:0000271', 'HP:0000322'

HP:0032053 {'HP:0000001', 'HP:0012443', 'HP:0000707', 'HP:0032053', 'HP:0000118', 'HP:0012639', 'HP:0002011', 'HP:0002060', 'HP:0032046', 'HP:0002539', 'HP:0002538', 'HP:0032051', 'HP:0100547'} {'HP:0000707'}
HP:0010289 {'HP:0010289', 'HP:0000001', 'HP:0000152', 'HP:0000118', 'HP:0031816', 'HP:0000174', 'HP:0000163', 'HP:0410005', 'HP:0000175', 'HP:0000202', 'HP:0000153', 'HP:0000271', 'HP:0100737', 'HP:0000234'} {'HP:0000152'}
HP:0007374 {'HP:0002134', 'HP:0000707', 'HP:0000118', 'HP:0002339', 'HP:0012639', 'HP:0007374', 'HP:0012444', 'HP:0007369', 'HP:0002977', 'HP:0100547', 'HP:0000001', 'HP:0012443', 'HP:0010994', 'HP:0007367', 'HP:0002011', 'HP:0010993', 'HP:0002060'} {'HP:0000707'}
HP:0011051 {'HP:0000164', 'HP:0000001', 'HP:0006483', 'HP:0000152', 'HP:0000118', 'HP:0031816', 'HP:0000163', 'HP:0011051', 'HP:0009804', 'HP:0000271', 'HP:0000153', 'HP:0001592', 'HP:0011076', 'HP:0000234'} {'HP:0000152'}
HP:0032052 {'HP:0000001', 'HP:0012443', 'HP:0000707', 'HP:0000118', 'HP:0012639'

In [18]:
result

,HPO_ID,Definition,Ancestor
0,HP:0025461,Any anomaly of cell structure.,HP:0025354
1,HP:0031352,An unpleasant sensation of tightness or pressu...,HP:0025142
2,HP:0000464,An abnormality of the neck.,HP:0000152
3,HP:0031863,The presence of an infectious agent in the blo...,HP:0001939
4,HP:0001928,An abnormality of the process of blood coagula...,HP:0001871
...,...,...,...
7939,HP:0011046,Agenesis of upper central primary incisor.,HP:0000152
7940,HP:0011047,Agenesis of lower primary incisor.,HP:0000152
7941,HP:0011045,Agenesis of upper secondary incisor.,HP:0000152
7942,HP:0011048,Agenesis of lower secondary incisor.,HP:0000152


In [19]:
result.to_csv("Single_ancestor.csv", index= False)

In [34]:
temp = int(result[result['HPO_ID']=='HP:0025461'].index.values)
print (temp)


0


In [21]:
result['HPO_ID']

0       HP:0025461
1       HP:0031352
2       HP:0000464
3       HP:0031863
4       HP:0001928
           ...    
7939    HP:0011046
7940    HP:0011047
7941    HP:0011045
7942    HP:0011048
7943    HP:0011050
Name: HPO_ID, Length: 7944, dtype: object